[View in Colaboratory](https://colab.research.google.com/github/facelesshawk/hello-worldFaceless/blob/master/tom_jerry_bestModel.ipynb)

In [0]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from keras.preprocessing import image
import numpy as np
from skimage.transform import resize

In [72]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [76]:
cd drive

/content/drive


In [2]:
count = 0
videoFile = "Tom and jerry.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [3]:
count = 0
videoFile = "Tom and Jerry 3.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:

# 2. Load a file by ID and create local file.
downloaded = drive.CreateFile({'id':'1AI9fNwD5qQ9XcAe3MZP4ycV88c7p2NSk'})
# replace fileid with Id of file you want to access
downloaded1 = drive.CreateFile({'id':'1zCMfZrgKNtLOHnBZ6MmyRDlM1gSL6yoC'})
downloaded.GetContentFile('mapping(1).csv') # now you can use export.csv
downloaded1.GetContentFile('testing(1).csv')

data = pd.read_csv('mapping(1).csv')
test = pd.read_csv('testing(1).csv')

In [0]:
X = []
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)
X = np.array(X)

In [0]:
test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [0]:
from keras.utils import np_utils
train_y = np_utils.to_categorical(data.Class)
test_y = np_utils.to_categorical(test.Class)

In [111]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224,3)).astype(int)
    image.append(a)
X = np.array(image)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [112]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [0]:
from keras.applications.vgg16 import preprocess_input
X = X.astype('float32')
X = preprocess_input(X)
test_image = test_image.astype('float32')
test_image = preprocess_input(test_image)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, train_y, test_size=0.3, random_state=42)

In [115]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
test_image = base_model.predict(test_image)
X_train = X_train.reshape(208, 7*7*512)
X_valid = X_valid.reshape(90, 7*7*512)
test_image = test_image.reshape(186, 7*7*512)
train = X_train/X_train.max()
X_valid = X_valid/X_train.max()
test_image = test_image/test_image.max()
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid'))   # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=512, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=256, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(3, activation='sigmoid'))            # output layer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
class_weights = compute_class_weight('balanced',np.unique(data.Class), data.Class)  # computing weights of different classes
from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]      # model check pointing based on validation loss
model.fit(train, y_train, epochs=100,verbose=1, validation_data=(X_valid, y_valid), class_weight=class_weights, callbacks=callbacks_list)

Train on 208 samples, validate on 90 samples
Epoch 1/100
208/208 [==============================] - 1s 5ms/step - loss: 1.1480 - acc: 0.3077 - val_loss: 1.0883 - val_acc: 0.3889

Epoch 00001: val_loss improved from inf to 1.08831, saving model to weights.best.hdf5
Epoch 2/100
208/208 [==============================] - 0s 1ms/step - loss: 1.0821 - acc: 0.3894 - val_loss: 1.0829 - val_acc: 0.3889

Epoch 00002: val_loss improved from 1.08831 to 1.08295, saving model to weights.best.hdf5
Epoch 3/100
208/208 [==============================] - 0s 1ms/step - loss: 1.0833 - acc: 0.3894 - val_loss: 1.0769 - val_acc: 0.3889

Epoch 00003: val_loss improved from 1.08295 to 1.07691, saving model to weights.best.hdf5
Epoch 4/100
208/208 [==============================] - 0s 2ms/step - loss: 1.0660 - acc: 0.4087 - val_loss: 1.0696 - val_acc: 0.3889

Epoch 00004: val_loss improved from 1.07691 to 1.06963, saving model to weights.best.hdf5
Epoch 5/100
208/208 [==============================] - 0s 1ms/s

In [116]:
model.load_weights("weights.best.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
scores = model.evaluate(test_image, test_y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

186/186 [==============================] - 0s 995us/step
acc: 59.14%


In [117]:
!ls

 frame0.jpg     frame20.jpg    frame51.jpg   test161.jpg
 frame100.jpg   frame210.jpg   frame52.jpg   test162.jpg
 frame101.jpg   frame211.jpg   frame53.jpg   test163.jpg
 frame102.jpg   frame212.jpg   frame54.jpg   test164.jpg
 frame103.jpg   frame213.jpg   frame55.jpg   test165.jpg
 frame104.jpg   frame214.jpg   frame56.jpg   test166.jpg
 frame105.jpg   frame215.jpg   frame57.jpg   test167.jpg
 frame106.jpg   frame216.jpg   frame58.jpg   test168.jpg
 frame107.jpg   frame217.jpg   frame59.jpg   test169.jpg
 frame108.jpg   frame218.jpg   frame5.jpg    test16.jpg
 frame109.jpg   frame219.jpg   frame60.jpg   test170.jpg
 frame10.jpg    frame21.jpg    frame61.jpg   test171.jpg
 frame110.jpg   frame220.jpg   frame62.jpg   test172.jpg
 frame111.jpg   frame221.jpg   frame63.jpg   test173.jpg
 frame112.jpg   frame222.jpg   frame64.jpg   test174.jpg
 frame113.jpg   frame223.jpg   frame65.jpg   test175.jpg
 frame114.jpg   frame224.jpg   frame66.jpg   test176.jpg
 frame115.jpg   frame225.jpg   f